# Topic Categorization with PySS3
<br>
<div style="text-align:right"><i>To open and run this notebook <b>online</b>, click here: <a href="https://mybinder.org/v2/gh/sergioburdisso/pyss3/master?filepath=examples/topic_categorization.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" style="display: inline"></a></i></div>

This is the notebook for the ["Topic Categorization"](https://pyss3.readthedocs.io/en/latest/tutorials/topic-categorization.html) tutorial. In this notebook, we will see how we can use the [PySS3](https://github.com/sergioburdisso/pyss3) Python package to deploy models for Sentiment Analysis on Movie Reviews.

Let us begin! First, we need to import the modules we will be using:

---

Before we begin, let's import needed modules...

In [ ]:
%matplotlib inline

from pyss3 import SS3
from pyss3.util import Dataset, Evaluation, span
from pyss3.server import Live_Test

from sklearn.metrics import accuracy_score

... and unzip the "movie_review.zip" dataset inside the `datasets` folder.

In [ ]:
!unzip -u datasets/topic.zip -d datasets/

Ok, now we are ready to begin. Let's create a new SS3 instance

In [ ]:
clf = SS3()

What are the default [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values? let's see

In [ ]:
s, l, p, _ = clf.get_hyperparameters()

print("Smoothness(s):", s)
print("Significance(l):", l)
print("Sanction(p):", p)

Ok, now let's load the training and the test set using the `load_from_files` function from `pyss3.util`. Since, in this dataset, there's a single file for each category, we will use the argument ``folder_label=False`` to tell PySS3 to use each file as a different category and each line inside of it as a different document:

In [ ]:
x_train, y_train = Dataset.load_from_files("datasets/topic/train", folder_label=False)
x_test, y_test = Dataset.load_from_files("datasets/topic/test", folder_label=False)

Let's train our model...


In [ ]:
# we could've have used clf.fit(x_train, y_train) here, they're equivalent!
# We decided to use `clf.train` only because is more "user-friendly".

clf.train(x_train, y_train)  # clf.fit(x_train, y_train)

Note that we don't have to create any document-term matrix! we are using just the plain `x_train` documents :D cool uh? SS3 learns a (spacial kind of) language model for each category and therefore it doesn't need to create any document-term matrices :) in fact, the very concept of "document" becomes irrelevant...

Now that the model has been trained, let's test it using the documents in `x_test`. First, we will do it "in the sklearn's own way" with:

In [ ]:
y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_pred, y_test)

print("Accuracy was:", accuracy)

Alternatively, we could've done it "in the PySS3's own way" by using the built-in ``test`` function provided by the ``Evaluation`` class that we have imported from ``pyss3.util`` at the beginning of this notebook, as follows:

In [ ]:
Evaluation.test(clf, x_test, y_test)

The advantage of using this built-in function is that with just a single line of code we get:
* The performance measured in terms of all the well-known metrics ('accuracy', 'precision', 'recall', and 'f1-score').
* A plot showing the obtained confusion matrix, and...
* Since all the evaluations performed using the ``Evaluation`` class are permanently cached, if we ever perform this test again, the evaluation will be skipped and values will be retrieved from the cache storage (saving us a lot of time! especially when performing long evaluations).

As we can see, the performance doesn't look bad using the default [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values, however, let's now manually analyze what our model has actually learned by using the interactive "live test". Note that since we are not going to use the `x_test` for this live test**(\*)** but instead the documents in "datasets/topic/live_test", we must use the `set_testset_from_files` method to tell the Live Test to load documents from there instead.

**(\*)** *try it if you want but since `x_test` contains (preprocessed) tweets, they don't look really good and clean.*

In [ ]:
# Live_Test.run(clf, x_test, y_test)  # <- this visualization doesn't look really clean and good so, instead,
                                      #    we will use the documents in "live_test" folder:

Live_Test.set_testset_from_files("datasets/topic/live_test")

Live_Test.run(clf)  # <- Unfortunately, if you're running the notebook online with 
                    #    Binder, won't work, sorry :(

<p style="color:red">(!) To <b>STOP</b> the server, press <b>Esc</b> once and then the <b>i</b> key twice</p>
<p style="color:red"><u>NOTE</u>: Unfortunately, the Live Test will ONLY WORK if you run this notebook, locally, on your computer. Therefore, if you're using the online Binder version, won't work, sorry  :( ... a Live Test like <a href="http://tworld.io/ss3/live_test_online/#30303" target="_blank">this one</a> would have been opened up, locally, in your browser but using the model you just trained above.</p>

Makes sense to you? (remember you can use the options to select "words" as the Description Level if you want to know based on what words, and to what degree, is making classification decisions)

Live test doesn't look bad, however,there are a couple of documents that are clearly misclassified. We will try creating a "more intelligent" version of this model, a version that can recognize variable-length word n-grams "on the fly". Thus, when calling the `train` we will pass an extra argument `n_grams=3` to indicate we want SS3 to learn to recognize important words, bigrams, and 3-grams _(If you're curious and want to know how this is actually done by SS3, read the paper "t-SS3: a text classifier with dynamic n-grams for early risk detection over text streams", preprint available [here](https://arxiv.org/abs/1911.06147))_.

In [ ]:
clf = SS3()

clf.train(x_train, y_train, n_grams=3)  # <-- note the n_grams=3 argument here

Now let's see if the performance has improved...

In [ ]:
y_pred = clf.predict(x_test)
print("Accuracy:", accuracy_score(y_pred, y_test))

Yeah, the accuracy slightly improved but more importantly, we should now see that the model has learned "more intelligent patterns" involving sequences of words when using the interactive "live test" (like "machine learning", "artificial intelligence", "self-driving cars", etc. for the "science&technology" category). Let's see...

In [ ]:
Live_Test.run(clf)  # <- remember it won't work online using Binder :(

# (!) Remember: to STOP the server, press `Esc` once and then the `I` key twice

Fortunately, our model has learned to recognize these important sequences (such as "artificial intelligence" and "machine learning" in doc_2.txt,  "self-driving cars" in doc_6.txt, etc.). However, again, some documents aren’t perfectly classified, for instance, doc_10.txt was classified as “science&technology” (as the second topic) which is clearly wrong...

So, one last thing we are going to do is to try to find better [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values to improve our model's performance. For example, the following values will improve our classification performance:

In [ ]:
clf.set_hyperparameters(s=0.32, l=1.62, p=2.35)

Let's see if it's true...

In [ ]:
Evaluation.test(clf, x_test, y_test)

The accuracy has improved as expected and the confusion matrix look much better now :)

Finally, we could take a look at what our final model looks like using the Live Test tool one last time.

In [ ]:
Live_Test.run(clf)

# (!) Remember: to stop the server, press `Esc` and then the `I` key twice

**Want to know how we found out those hyperparameter values** were going to improve our classifier accuracy? Just read the next section! ;)

---
## Hyperparameter Optimization

In this section we will see how we can use PySS3's ``Evaluation`` class to perform [Hyperparameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization), which allows us to find better hyperparameter values for our models. To do this, we will perform [grid searches](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Grid_search) using the [Evaluation.grid_search()](https://pyss3.rtfd.io/en/latest/api/index.html#pyss3.util.Evaluation.grid_search) function.

Let's create a new (standard) instance of the SS3 classifier. This will speed things up because the model we currently have in ``clf`` recognize variable-length word n-grams, the grid search won't run as fast as with a (standard) model that recognize only words (and the same "best" hyperparameter values usually work for both of them). Note: just ignore the (optional) ``name`` argument below, we're giving our model the name "topics" only to make things clearer when we create the interactive 3D evaluation plot.

In [ ]:
clf = SS3(name="topics")

clf.train(x_train, y_train)

The [Evaluation.grid_search()](https://pyss3.rtfd.io/en/latest/api/index.html#pyss3.util.Evaluation.grid_search) takes, for each hyperparameter, the list of values to use in the search, for instance ``s=[0.25, 0.5, 0.75, 1]`` indicates you want the ``grid_search`` to try out evaluating the classifier using those 4 values for the sigma (``s``) hyperparameter. However, for simplicity, instead of using a manually crafted long list of values, we will use the ``span`` function we have imported from ``pyss3.util`` at the beginning of this notebook. This function will create a list of values for us, giving a lower and upper bound, and the number of elements to be generated. For instance, if we want a list of 6 numbers between 0 and 1, we could use:

In [ ]:
span(0, 1, 6)

Thus, we will use the following values for each of the three hyperparameters:

In [ ]:
s_vals=span(0.2, 0.8, 6)  # [0.2 , 0.32, 0.44, 0.56, 0.68, 0.8]
l_vals=span(0.1, 2, 6)    # [0.1 , 0.48, 0.86, 1.24, 1.62, 2]
p_vals=span(1.75, 2.75, 6)    # [1.75, 1.95, 2.15, 2.35, 2.55, 2.75]

To speed things up, unlike in the ["Movie Reviews (Sentiment Analysis)"](https://pyss3.readthedocs.io/en/latest/tutorials/movie-review.html) tutorial, we will perform the grid search using only the test set (we won't use k-fold cross-validation). Once the search is over, ``Evaluation.grid_search`` will return the hyperparamter values that obtained the best accuracy for us.

In [ ]:
# the search should take about 15 minutes
best_s, best_l, best_p, _ = Evaluation.grid_search(
    clf, x_test, y_test,
    s=s_vals, l=l_vals, p=p_vals
)

In [ ]:
print("The hyperparameter values that obtained the best Accuracy are:")
print("Smoothness(s):", best_s)
print("Significance(l):", best_l)
print("Sanction(p):", best_p)

And that's how we found out that these hyperparameter values (``s=0.32, l=1.62, p=2.35``) were going to improve our classifier accuracy.

---
**NOTE:** what if we want to find hyperparameter values that performed the best but using a different metric other than accuracy? for example, what if we wanted to find the hyperparameter values that will improve the precision for the ``sports`` topic? we can use the ``Evaluation.get_best_hyperparameters()`` function as follows:

In [ ]:
s, l, p, _ = Evaluation.get_best_hyperparameters(metric="precision", metric_target="sports")

print("s=%.2f, l=%.2f, and p=%.2f" % (s, l, p))

Or the macro averaged f1 score?

In [ ]:
s, l, p, _ = Evaluation.get_best_hyperparameters(metric="f1-score", metric_target="macro avg")

print("s=%.2f, l=%.2f, and p=%.2f" % (s, l, p))

Alternatively, we could have also added these 2 arguments, metric and target, to the grid search in the first place :) (e.g. ``Evaluation.grid_search(..., metric="f1-score", metric_target="macro avg")``).

Note that this ``get_best_hyperparameters`` function gave us the values right away! this is because instead of performing the grid search again, this function uses the evaluation cache to retrieve the best values from disk, which save us a lot of time!

---

### Interactive 3D Evaluation Plot

The ``Evaluation`` class comes with a really useful function, ``Evaluation.plot()``, that we can use to create an interactive 3D evaluation plot (We highly recommend reading this [brief section](https://pyss3.rtfd.io/en/latest/user_guide/visualizations.html#evaluation-plot), from the documentation, in which it is briefly described). Instead of using the single value returned from the ``Evaluation.grid_search()`` we could use this plot to have a broader view of the relationship between the different hyperparameter values and the performance of our model in the task being addressed. The ``Evaluation.plot()`` function creates a portable HTML file containing the interactive plot for us, and then opens it up in your browser. Let's give it a shot:

In [ ]:
Evaluation.plot()

---
**NOTE:** If your running this notebook online using Binder, the plot won't open. Fortunately, this time there's a work around! if we list the files in the current directory:

In [ ]:
!ls

We can see the plot file has been actually created with the name we have given to our model in it, ``ss3_model_evaluation[topics].html``, however, since the Jupyter kernel is not running in your computer, PySS3 was not able to open the plot for you in your browser.

**Work around:** Go to the "File" menu (upper-left corner) and then select the "Open..." option, then click on this file to manually open it up in your browser (or... just click <a href="/view/ss3_model_evaluation%5Btopics%5D.html" target="_blank">here</a>!)

---

You should see a plot like this one:

![](imgs/topic_evaluations.png)

Rotate the camera and move the cursor over the pink point, that is the point that obtained the best performance (in terms of accuracy), that is, this is the point that the ``grid_search`` gave us. Among the information that is displayed when moving the cursor over the points, a compact version of the obtained confusion matrix is shown. Feel free to play a little bit with this interactive 3D evaluation plot, for instance try changing the metric and target from the options panel :D